In [1]:


import numpy as np
import random

In [2]:
# Define the maze environment
# 0 = Free space, 1 = Obstacle, 2 = Goal
maze = np.array([
    [0, 1, 0, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 1, 0, 0],
    [0, 0, 0, 1, 2]
])

# Define possible actions
# 0 = Up, 1 = Right, 2 = Down, 3 = Left
actions = [0, 1, 2, 3]
action_dict = {
    0: (-1, 0),  # Move up
    1: (0, 1),   # Move right
    2: (1, 0),   # Move down
    3: (0, -1)   # Move left
}

# Initialize Q-table
q_table = np.zeros((maze.shape[0], maze.shape[1], len(actions)))

# Parameters for Q-learning
alpha = 0.1    # Learning rate
gamma = 0.9    # Discount factor
epsilon = 0.9  # Exploration rate
max_episodes = 1000
max_steps = 100

# Rewards
goal_reward = 100
step_penalty = -1
obstacle_penalty = -100

# Helper function to get next state based on action
def get_next_state(state, action):
    row, col = state
    next_row = row + action_dict[action][0]
    next_col = col + action_dict[action][1]
    
    if next_row < 0 or next_row >= maze.shape[0] or next_col < 0 or next_col >= maze.shape[1]:
        return state  # Stay in current state if out of bounds
    
    if maze[next_row, next_col] == 1:  # Hit an obstacle
        return state  # Stay in current state
    
    return (next_row, next_col)

# Q-learning algorithm
for episode in range(max_episodes):
    state = (0, 0)  # Start at the top-left corner of the maze
    
    for step in range(max_steps):
        if random.uniform(0, 1) < epsilon:  # Explore
            action = random.choice(actions)
        else:  # Exploit
            action = np.argmax(q_table[state[0], state[1]])
        
        next_state = get_next_state(state, action)
        
        # Assign rewards
        if maze[next_state[0], next_state[1]] == 2:  # Reached the goal
            reward = goal_reward
        elif maze[next_state[0], next_state[1]] == 1:  # Hit an obstacle
            reward = obstacle_penalty
        else:
            reward = step_penalty
        
        # Update Q-value
        old_q_value = q_table[state[0], state[1], action]
        next_max_q_value = np.max(q_table[next_state[0], next_state[1]])
        new_q_value = (1 - alpha) * old_q_value + alpha * (reward + gamma * next_max_q_value)
        q_table[state[0], state[1], action] = new_q_value
        
        # Move to the next state
        state = next_state
        
        # If goal is reached, end episode
        if maze[state[0], state[1]] == 2:
            break
    
    # Decrease epsilon (less exploration as training progresses)
    epsilon = max(0.1, epsilon * 0.99)

# Display the learned Q-values
print("Learned Q-table:")
print(q_table)

Learned Q-table:
[[[ 19.02421892  19.3566767   24.51916557  19.50598582]
  [  0.           0.           0.           0.        ]
  [ 45.96031678  54.9539      41.21763794  46.22091106]
  [ 53.29204596  62.171       53.44184636  47.50152538]
  [ 60.99174061  61.83867824  70.19        53.04785884]]

 [[ 19.93022202  20.07397646  28.35462841  22.95263404]
  [  0.           0.           0.           0.        ]
  [ 48.45851     41.60007947  35.75459646  41.72315393]
  [  0.           0.           0.           0.        ]
  [ 60.30534161  69.0692682   79.1         66.243028  ]]

 [[ 22.99098008  32.61625379  18.82525514  25.90087221]
  [ 30.76047262  37.3513931   31.27773352  26.82566999]
  [ 42.612659    36.09049172  35.77431115  32.10383993]
  [  0.           0.           0.           0.        ]
  [ 67.33626227  78.3274411   89.          77.99812955]]

 [[ 26.83824509  -2.24762432  -4.6932054   -3.30122217]
  [  0.           0.           0.           0.        ]
  [  0.           0.     

In [4]:
# Display the optimal policy (best actions for each state)
optimal_policy = np.zeros(maze.shape, dtype=str)
for i in range(maze.shape[0]):
    for j in range(maze.shape[1]):
        if maze[i, j] == 1:
            optimal_policy[i, j] = 'X'  # Obstacle
        elif maze[i, j] == 2:
            optimal_policy[i, j] = 'G'  # Goal
        else:
            action = np.argmax(q_table[i, j])
            if action == 0:
                optimal_policy[i, j] = '↑'
            elif action == 1:
                optimal_policy[i, j] = '→'
            elif action == 2:
                optimal_policy[i, j] = '↓'
            elif action == 3:
                optimal_policy[i, j] = '←'

# Mark the optimal path with '*'
def display_optimal_policy_with_path(policy, maze):
    state = (0, 0)  # Start at the top-left corner
    path_maze = np.copy(policy)
    
    while maze[state[0], state[1]] != 2:  # Continue until the goal is reached
        action = policy[state[0], state[1]]
        if action in ['↑', '→', '↓', '←']:
            path_maze[state[0], state[1]] = '*'  # Mark the path with '*'
        
        if action == '↑':
            next_state = (state[0] - 1, state[1])
        elif action == '→':
            next_state = (state[0], state[1] + 1)
        elif action == '↓':
            next_state = (state[0] + 1, state[1])
        elif action == '←':
            next_state = (state[0], state[1] - 1)
        
        state = next_state  # Move to the next state
    
    # Mark the goal with 'G'
    path_maze[state[0], state[1]] = 'G'
    
    for row in path_maze:
        print(" ".join(row))

# Example usage:
print("\nOptimal Policy with path marked by '*':")
display_optimal_policy_with_path(optimal_policy, maze)



Optimal Policy with path marked by '*':
* X * * *
* X * X *
* * * X *
↑ X X → *
↑ → ↑ X G
